In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
#Cuartiles
from pyspark.sql import functions as F

def categorize_by_quintiles(df, column_names, filter_condition, quintile_probabilities=[0.25, 0.5, 0.75, 1.0], relative_error=0.01):
    for column in column_names:
        # Filtrar y calcular los quintiles
        quintiles = df.filter(filter_condition).approxQuantile(column, quintile_probabilities, relative_error)
        Minimo = df.agg({column: "min"}).collect()[0][0]
        
        # Crear la nueva columna categorizada
        df = df.withColumn(
            f'{column}_Q',
            F.when(F.col(column) <= quintiles[0], f"1. {format(Minimo, '.3f')} - {format(quintiles[0], '.3f')}")
             .when(F.col(column) <= quintiles[1], f"2. {format(quintiles[0], '.3f')} - {format(quintiles[1], '.3f')}")
             .when(F.col(column) <= quintiles[2], f"3. {format(quintiles[1], '.3f')} - {format(quintiles[2], '.3f')}")
             .when(F.col(column) <= quintiles[3], f"4. {format(quintiles[2], '.3f')} - {format(quintiles[3], '.3f')}")
             .otherwise("98. Missing")
        )
    return df

In [0]:
#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
from collections import Counter

def find_repeated_columns(df):
    column_names = df.columns
    column_counts = Counter(column_names)
    repeated_columns = [col for col, count in column_counts.items() if count > 1]
    return repeated_columns

In [0]:
def find_missing_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    
    missing_in_df2 = columns_df1 - columns_df2
    missing_in_df1 = columns_df2 - columns_df1
    
    print("Columnas en df1 pero no en df2:", missing_in_df2)
    print("Columnas en df2 pero no en df1:", missing_in_df1)

In [0]:
def generar_codmes(inicio, final):
    codmes_list = []
    anio_inicial = inicio // 100
    mes_inicial = inicio % 100
    
    anio_final = final // 100
    mes_final = final % 100
    
    anio = anio_inicial
    mes = mes_inicial
    
    # Generar la lista hasta el mes final
    while (anio < anio_final) or (anio == anio_final and mes <= mes_final):
        codmes_list.append(anio * 100 + mes)
        mes += 1
        if mes > 12:
            mes = 1  # Pasar a enero del siguiente año
            anio += 1
    return codmes_list

In [0]:
from pyspark.sql.functions import coalesce, lit

def dias_atraso(base, identificador, inicio, final): 
    df_list = []
    espacio = generar_codmes(inicio = inicio, final = final)
    for codmes in espacio:
        numeromes = (codmes // 100) * 12 + (codmes % 100)
        codmes_f1 = ((numeromes - 1 + 1) // 12) * 100 + ((numeromes + 11 - 11 * 1) % 12) + 1
        codmes_f2 = ((numeromes - 1 + 2) // 12) * 100 + ((numeromes + 11 - 11 * 2) % 12) + 1
        codmes_f3 = ((numeromes - 1 + 3) // 12) * 100 + ((numeromes + 11 - 11 * 3) % 12) + 1

        df_base = base.filter(f"CODMES = {codmes}").select("CODMES", identificador, "CTDMESMADURACION", "MTOSALDOCAPITALSOL","TIPBLOQUEOPRODUCTO", "CTDDIAATRASO")
        df_f1 = base.filter(f"CODMES = {codmes_f1}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f2 = base.filter(f"CODMES = {codmes_f2}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f3 = base.filter(f"CODMES = {codmes_f3}").select("CODMES", identificador, "CTDDIAATRASO")

        df_parcial = df_base.alias('A') \
                .join(df_f1.alias('B'), on = identificador, how = 'left') \
                .join(df_f2.alias('C'), on = identificador, how = 'left') \
                .join(df_f3.alias('D'), on = identificador, how = 'left') \
                .selectExpr('A.*', 
                            'COALESCE(B.CTDDIAATRASO, 0) AS CTDDIAATRASO_F1', 
                            'COALESCE(C.CTDDIAATRASO, 0) AS CTDDIAATRASO_F2', 
                            'COALESCE(D.CTDDIAATRASO, 0) AS CTDDIAATRASO_F3')
                
        df_list.append(df_parcial)
        print(f'Iteración {codmes}:', f't0:{codmes}', f't1:{codmes_f1}', f't2:{codmes_f2}', f't3:{codmes_f3}')

    df_final = df_list[0]
    for df in df_list[1:]:
        df_final = df_final.unionByName(df)

    return df_final

In [0]:
def rezago_codmes_col(base, rezago_mes):
    for i in range(0, rezago_mes + 1, 1):
        base = base.withColumn(f"CODMES_{i}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), -i), "yyyyMM"))
        base = base.withColumn(f"CODMES_{i}", col(f"CODMES_{i}").cast("integer"))
    return base

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

In [0]:
path_CSV = "abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202503/MM_APPSCORE_GARANTIAHIP"
 
spark.sql("""DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_GARANTIAHIP_202503;""")
 
spark.sql(f"""
    CREATE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_GARANTIAHIP_202503
    USING CSV
    OPTIONS (header = "true", inferSchema = "true",delimiter = ";")    
    LOCATION '{path_CSV}'
""")

In [0]:
hm_mtz_adm_hipotecario_driver = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_hm_mtz_admision_hipotecario_202503_F')

In [0]:
hm_adm_gahi_001 = hm_mtz_adm_hipotecario_driver.withColumn(
    "FLG_FINALIDAD",
    when(col("DESTIPFINALIDADCREDHIPOTECARIO").isNull(), lit(1))
    .when(col("DESTIPFINALIDADCREDHIPOTECARIO").isin(
        "LIBRE DISPONIBILIDAD",
        "COMPRA VENTA DE LOCAL COMERCIAL",
        "C-V/REMODEL/AMPLI DE LOCAL COMERCIAL",
        "C-V Oficinas/Locales BT",
        "LIBRE DISP. CON GARANT.LOCAL COMERC.",
        "MIGRACION DE OTRO BANCO"
    ), lit(1))
    .when((col("CODMES") >= 202307) & (col("DESTIPFINALIDADCREDHIPOTECARIO") == "COMPRA/VENTA BIEN TERMINADO"), lit(1))
    .otherwise(lit(0))
).filter(col("TIPCREDITO").isin('GAHI', 'GAHP', 'GAHR', 'GAPC'))

In [0]:
hm_adm_gahi_001.createOrReplaceTempView("temp_hm_adm_gahi_001")

In [0]:
base_espejo=spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_GARANTIAHIP_202503")

In [0]:
base_espejo.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = base_espejo.groupBy( "CODSOLICITUD").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
base_espejo.createOrReplaceTempView("temp_base_espejo")

In [0]:
hm_adm_gahi_002 = spark.sql("""
    SELECT A.*, 
           /*VARIABLES CALIBRADO*/
           D.XB_GAHI,
           D.XB_GAHI_PRELIMINAR,
           D.XB_NUE_APP_CEF, /*89%*/
           D.XB_MODUL_GAHI, /*11%*/
           D.LTV_NEW AS LTV_NEW_CAL,
           D.LTV_NEW_A2,
           D.CTDPLAZOAPROBADO_TP,
           D.CTDPLAZOAPROBADO_A2,
           D.RIESGO_ZONA2,
           D.RIESGO_ZONA2_ST,
           D.INGRESO_SOL_CONY_TP,
           D.INGRESO_SOL_CONY_D3, 
           D.FLG_MONTO AS FLG_MONTO_CAL, /*SI >=325,000 FLG_MONTO=1*/
           D.PD_GAHI AS PD_GAHI_ESPEJO
    FROM temp_hm_adm_gahi_001 A
    LEFT JOIN temp_base_espejo D ON A.CODSOLICITUD=D.CODSOLICITUD;
""")

In [0]:
hm_adm_gahi_002.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hm_adm_gahi_002.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
# Add the TOTAL column
hm_adm_gahi = hm_adm_gahi_002.withColumn("TOTAL", lit('1'))

# Add the PAUTA column
hm_adm_gahi = hm_adm_gahi.withColumn("PAUTA", when(col("SC_GAHI") >= 345, 1).otherwise(0))

# Add the RSC column
hm_adm_gahi = hm_adm_gahi.withColumn("RSC", when(col("SC_GAHI").isNull(), '99. Missing')
                   .when(col("SC_GAHI") < 385, '1. <0 - 385>')
                   .when(col("SC_GAHI") < 415, '2. [385 - 415>')
                   .when(col("SC_GAHI") < 450, '3. [415 - 450>')
                   .when(col("SC_GAHI") < 495, '4. [450 - 495>')
                   .when(col("SC_GAHI") >= 495, '5. [495 - +++>')
                   .otherwise('98. Otro'))

# Add the RPD column
hm_adm_gahi = hm_adm_gahi.withColumn("RPD", when(col("PD_GAHI").isNull(), '99. Missing')
                   .when(col("PD_GAHI") <= 0.005, '1. <0 - 0.5%]')
                   .when(col("PD_GAHI") <= 0.010, '2. <0.5 - 1.0%]')
                   .when(col("PD_GAHI") <= 0.015, '3. <1.0 - 1.5%]')
                   .when(col("PD_GAHI") <= 0.025, '4. <1.5 - 2.5%]')
                   .when(col("PD_GAHI") > 0.025, '5. <2.5 - ++>')
                   .otherwise('98. Otro'))

# Add the MONTO_R column
hm_adm_gahi = hm_adm_gahi.withColumn("MONTO_R", when(col("MTOAPROBADO_SOLES").isNull(), '99. Missing')
                   .when(col("MTOAPROBADO_SOLES") <= 87500, '1. [0 - 87,500]')
                   .when(col("MTOAPROBADO_SOLES") <= 140000, '2. <87,500 - 140,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 215000, '3. <140,000 - 215,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 300000, '4. <215,000 - 300,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 600000, '5. <300,000 - 600,000]')
                   .otherwise('6. <600,000 - ++>'))

# Add the MISS_PD column
hm_adm_gahi = hm_adm_gahi.withColumn("MISS_PD", when(col("PD_GAHI").isNull(), 1).otherwise(0))

# Add the MISS_MONTO column
hm_adm_gahi = hm_adm_gahi.withColumn("MISS_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), 1).otherwise(0))

# Add the RAN_INGR_CY column
hm_adm_gahi = hm_adm_gahi.withColumn("RAN_INGR_CY", when(col("INGRESO_SOL_CONY_TP").isNull(), '99. Missing')
                   .when(col("INGRESO_SOL_CONY_TP") <= 2500, '1. [0 - 2,500]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 5000, '2. <2,500 - 5,000]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 7500, '3. <5,000 - 7,500]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 10000, '4. <7,500 - 10,000]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 20000, '5. <10,000 - 20,000]')
                   .otherwise('6. <20,000 - ++>'))

# Add the FLG_MONTO_R column
hm_adm_gahi = hm_adm_gahi.withColumn("FLG_MONTO_R", when(col("FLG_MONTO_CAL").isNull(), '99. Missing')
                   .when(col("FLG_MONTO_CAL") == 1, '1. >=325,000')
                   .otherwise('2. <325,000'))

# Filter the dataframe
hm_adm_gahi = hm_adm_gahi.filter(col("FLG_FINALIDAD") == 1)


In [0]:
hm_adm_gahi.createOrReplaceTempView("temp_view")
spark.sql("CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_GAHI_202503_F LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202503/T45988_HM_ADM_HIP_GAHI_202503_F' AS SELECT * FROM temp_view")